# Treinamento do Modelo Naive Bayes
## Classificação de Acidentes de Trânsito

Este notebook treina um modelo de classificação usando **Naive Bayes** para identificar:
- Acidentes de trânsito graves
- Acidentes de trânsito moderados  
- Não acidentes

## 1. Importar Bibliotecas

In [5]:
import os
import cv2
import numpy as np
import pickle
from pathlib import Path
from skimage.feature import hog, local_binary_pattern
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

print("Bibliotecas importadas com sucesso!")

Bibliotecas importadas com sucesso!


## 2. Configurar Caminhos e Mapeamento de Classes

In [6]:
# Caminho para o dataset
path_raiz = Path('dataset_finalized')

# Mapeamento das classes
classes_num = {
    "severe_accident": 0,
    "moderate_accident": 1,
    "no_accident": 2
}

print(f"Dataset: {path_raiz}")
print(f"Classes: {list(classes_num.keys())}")

Dataset: dataset_finalized
Classes: ['severe_accident', 'moderate_accident', 'no_accident']


## 3. Função de Extração de Features

Esta função extrai características das imagens usando:
- **HOG** (Histogram of Oriented Gradients): Detecta formas e contornos
- **Canny**: Densidade de bordas na imagem
- **Harris**: Densidade de cantos detectados
- **LBP** (Local Binary Pattern): Características de textura

In [7]:
def extrair_features_avancadas(caminho_img):
    """Extrai features da imagem usando HOG, bordas, cantos e textura"""
    img = cv2.imread(str(caminho_img), cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None

    img = cv2.resize(img, (128, 128))

    # HOG
    features_hog = hog(img, orientations=9, pixels_per_cell=(8, 8),
                       cells_per_block=(2, 2), visualize=False)

    # Bordas
    edges = cv2.Canny(img, 100, 200)
    densidade_bordas = np.array([np.sum(edges > 0) / edges.size])

    # Cantos
    dst = cv2.cornerHarris(img, 2, 3, 0.04)
    densidade_cantos = np.array([np.sum(dst > 0.01 * dst.max()) / dst.size])

    # Textura
    lbp = local_binary_pattern(img, 8, 1, method="uniform")
    hist_lbp, _ = np.histogram(lbp.ravel(), bins=10, range=(0, 10), density=True)

    return np.hstack([features_hog, densidade_bordas, densidade_cantos, hist_lbp])

print("Função de extração de features definida!")

Função de extração de features definida!


## 4. Carregar e Processar Imagens do Dataset

In [8]:
dados_lista = []
labels_lista = []

print("Iniciando extração de features do dataset...")

for folder_path in path_raiz.iterdir():
    if folder_path.is_dir():
        nome_categoria = folder_path.name.replace("dataset_final_", "")
        
        if nome_categoria in classes_num:
            rotulo = classes_num[nome_categoria]
            print(f"Processando pasta: {nome_categoria}")
            
            arquivos = list(folder_path.glob('*'))
            contador = 0
            
            for path_arquivo in arquivos:
                if path_arquivo.suffix.lower() in ['.png', '.jpg', '.jpeg']:
                    features = extrair_features_avancadas(path_arquivo)
                    if features is not None:
                        dados_lista.append(features)
                        labels_lista.append(rotulo)
                        contador += 1
            
            print(f"   {contador} imagens processadas")

print(f"\nTotal de imagens carregadas: {len(dados_lista)}")

Iniciando extração de features do dataset...
Processando pasta: moderate_accident
   240 imagens processadas
Processando pasta: no_accident
   240 imagens processadas
Processando pasta: severe_accident
   240 imagens processadas

Total de imagens carregadas: 720


## 5. Preparar Dados para Treinamento

In [9]:
X = np.array(dados_lista)
y = np.array(labels_lista)

print(f"Shape dos dados: {X.shape}")
print(f"Shape dos labels: {y.shape}")
print(f"Número de features: {X.shape[1]}")

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nDados divididos:")
print(f"   Treino: {X_train.shape[0]} amostras")
print(f"   Teste: {X_test.shape[0]} amostras")

Shape dos dados: (720, 8112)
Shape dos labels: (720,)
Número de features: 8112

Dados divididos:
   Treino: 576 amostras
   Teste: 144 amostras


## 6. Normalizar Dados com StandardScaler

In [10]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Dados normalizados com StandardScaler")

Dados normalizados com StandardScaler


## 7. Treinar o Modelo Naive Bayes

In [11]:
modelo = GaussianNB()
modelo.fit(X_train, y_train)

print("Modelo treinado com sucesso!")

Modelo treinado com sucesso!


## 8. Avaliar o Modelo

In [12]:
y_pred = modelo.predict(X_test)

print("Relatório de classificação:")
print(classification_report(y_test, y_pred, target_names=list(classes_num.keys())))

Relatório de classificação:
                   precision    recall  f1-score   support

  severe_accident       0.72      0.82      0.77        40
moderate_accident       0.80      0.73      0.77        56
      no_accident       0.70      0.69      0.69        48

         accuracy                           0.74       144
        macro avg       0.74      0.75      0.74       144
     weighted avg       0.75      0.74      0.74       144



## 9. Salvar Modelo e Scaler

In [13]:
output_dir = Path('../website/ml_models')
output_dir.mkdir(parents=True, exist_ok=True)

model_path = output_dir / 'modelo_ia.pkl'
scaler_path = output_dir / 'scaler.pkl'

with open(model_path, "wb") as f:
    pickle.dump(modelo, f)

with open(scaler_path, "wb") as f:
    pickle.dump(scaler, f)

print(f"Modelo salvo em: {model_path}")
print(f"Scaler salvo em: {scaler_path}")
print(f"\nTreinamento concluído com sucesso!")

Modelo salvo em: ..\website\ml_models\modelo_ia.pkl
Scaler salvo em: ..\website\ml_models\scaler.pkl

Treinamento concluído com sucesso!
